In [1]:
import requests, json, sys, math, time, pickle, newspaper, cnfg
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from newspaper import Article
from nytimesarticle import articleAPI

In [12]:
import logging
#logging.basicConfig(filename='fox_error_log.log',filemode='w', level=logging.INFO)
logging.basicConfig(filename='nyt_error_log.log',filemode='w', level=logging.INFO)

In [10]:
nyt_api_config = cnfg.load(".nyt_api_config")
nyt_api_key = nyt_api_config['api_key']

## Functions to get URLs of refugee articles, and then pull those articles from APIs

In [9]:
def get_fox_urls():
    df = pd.DataFrame()
    for start in range(0, 2074, 10):  # 2074 articles match my criteria. API returns 10 at a time.
        url = 'http://api.foxnews.com/v1/content/search?q=refugees&min_date=2015-01-01&max_date=2015-12-31&type=article&start=' \
                    + str(start)
        response = requests.get(url)
        if response.status_code != 200:
            msg = 'Get response from start position %d failed: %d' % (start, response.status_code)
            logging.error(msg, exc_info=True)
        else:
            res = json.loads(response.text)
            df = df.append(pd.DataFrame.from_dict(res['response']['docs']), ignore_index=True)
        
    return df

def get_fox_text(urls, t):
    config=newspaper.Config()
    config.fetch_images = False
    
    for i in range(len(urls)):
        msg = 'Downloading url %d' % i
        logging.info(msg)
        a = Article(urls[i], config=config) 
        try:
            a.download()
            a.parse()
            t.append(a.text)
        except:
            t.append('')
            msg = 'Get text from %s failed:' % url[i]
            logging.error(msg, exc_info=True)
            
            
def get_nyt_urls():
    df = pd.DataFrame()
    api = articleAPI(nyt_api_key)
    for i in range(122): # 1579 articles match my criteria. The API only allows getting 1200.
        try:
            response = api.search(q = 'refugees', 
            fq = {'NOT type_of_material':'video',
                 'source':'The New York Times'}, 
                begin_date = 20150101,
                end_date = 20151231,
                fl = 'web_url,abstract,source,headline,pub_date,document_type,news_desk,type_of_material',
                page = i)
        except:
            msg = 'Get response from offset %d failed. Response status: %s' % (i, response['status'])
            logging.error(msg, exc_info=True)
        try:
            df = df.append(pd.DataFrame.from_dict(response['response']['docs']), ignore_index=True)
        except:
            msg = 'Appending response from offset %d failed. Response status: %s' % (i, response['status'])
            logging.error(msg, exc_info=True)
        time.sleep(1)
            
    return df


def get_nyt_text(urls, t):
    config=newspaper.Config()
    config.fetch_images = False
    
    for i in range(len(urls)):
        msg = 'Downloading url %d' % i
        logging.info(msg)
        a = Article(urls[i], config=config) 
        try:
            a.download()
            a.parse()
            t.append(a.text)
        except:
            t.append('')
            msg = 'Get text from %s failed:' % url[i]
            logging.error(msg, exc_info=True)
            

## Download Fox News articles

In [ ]:
fox_urls = get_fox_urls()

In [ ]:
fox_urls.url = fox_urls.url.apply(lambda x: x[0])

In [130]:
fox_urls.head()

,date,description,title,url
0,2015-11-25T13:58:12Z,<p>Bernie Goldberg analyzes how the media is c...,Sympathy for Syrian refugees,http://www.foxnews.com/transcript/2015/11/25/s...
1,2015-11-19T14:27:12Z,<p>Kerry Kennedy on how the left is handling f...,"Democrats, ISIS and refugees",http://www.foxnews.com/transcript/2015/11/19/d...
2,2015-12-03T18:47:56Z,!--StartFragment-->Sen.,Republicans: 12 terrorism-implicated refugees ...,http://www.foxnews.com/politics/2015/12/03/rep...
3,2015-11-29T11:37:14Z,Republican presidential candidate Ben Carson i...,Carson's Halloween candy solution for Syrian r...,http://www.foxnews.com/politics/2015/11/29/car...
4,2015-11-29T11:16:12Z,"!--StartFragment-->Asked by Chuck Todd on ""Mee...",Trump: Obama plans 250k refugees in America,http://www.foxnews.com/politics/2015/11/29/tru...


In [70]:
# with open('fox_urls.pkl', 'wb') as picklefile:
#     pickle.dump(fox_urls, picklefile)

In [ ]:
# with open('fox_urls.pkl', 'rb') as picklefile:
#     fox_urls = pickle.load(picklefile)

In [151]:
fox_text = []
get_fox_text(fox_urls.url, fox_text)

Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/newspaper/parsers.py", line 54, in fromstring
    cls.doc = lxml.html.fromstring(html)
  File "//anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 706, in fromstring
    doc = document_fromstring(html, parser=parser, base_url=base_url, **kw)
  File "//anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 600, in document_fromstring
    value = etree.fromstring(html, parser, **kw)
  File "lxml.etree.pyx", line 3032, in lxml.etree.fromstring (src/lxml/lxml.etree.c:68121)
  File "parser.pxi", line 1786, in lxml.etree._parseMemoryDocument (src/lxml/lxml.etree.c:102470)
  File "parser.pxi", line 1667, in lxml.etree._parseDoc (src/lxml/lxml.etree.c:101229)
  File "parser.pxi", line 1035, in lxml.etree._BaseParser._parseUnicodeDoc (src/lxml/lxml.etree.c:96139)
  File "parser.pxi", line 582, in lxml.etree._ParserContext._handleParseResultDoc (src/lxml/lxml.etree.c:91290)
  File "parser

In [ ]:
fox_urls['text'] = fox_text

In [ ]:
# with open('fox_text2.pkl', 'wb') as picklefile:
#     pickle.dump(fox_text, picklefile)
    
# with open('fox_urls2.pkl', 'wb') as picklefile:
#     pickle.dump(fox_urls, picklefile)

In [168]:
# with open('fox_urls2.pkl', 'rb') as picklefile:
#     fox_urls = pickle.load(picklefile)

## Download New York Times articles

In [13]:
nyt_urls = get_nyt_urls()
# The last 10 rows are from page 43. All the rest of the rows are in order of relevance.

In [156]:
# with open('nyt_urls.pkl', 'wb') as picklefile:
#     pickle.dump(nyt_urls, picklefile)

In [160]:
nyt_text = []
get_nyt_text(nyt_urls.web_url, nyt_text)

Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/newspaper/parsers.py", line 54, in fromstring
    cls.doc = lxml.html.fromstring(html)
  File "//anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 706, in fromstring
    doc = document_fromstring(html, parser=parser, base_url=base_url, **kw)
  File "//anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 600, in document_fromstring
    value = etree.fromstring(html, parser, **kw)
  File "lxml.etree.pyx", line 3032, in lxml.etree.fromstring (src/lxml/lxml.etree.c:68121)
  File "parser.pxi", line 1786, in lxml.etree._parseMemoryDocument (src/lxml/lxml.etree.c:102470)
  File "parser.pxi", line 1667, in lxml.etree._parseDoc (src/lxml/lxml.etree.c:101229)
  File "parser.pxi", line 1035, in lxml.etree._BaseParser._parseUnicodeDoc (src/lxml/lxml.etree.c:96139)
  File "parser.pxi", line 582, in lxml.etree._ParserContext._handleParseResultDoc (src/lxml/lxml.etree.c:91290)
  File "parser

In [163]:
nyt_urls['text'] = nyt_text

In [164]:
# with open('nyt_urls2.pkl', 'wb') as picklefile:
#     pickle.dump(nyt_urls, picklefile)
    
# with open('nyt_text2.pkl', 'wb') as pickletfile:
#     pickle.dump(nyt_text, picklefile)